This notebook explores methods for calculating MLD globally on llc4320 data

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import zarr
import numpy as np
import dask
import dask.array as da
import numcodecs

from fastjmd95 import jmd95numba 

import cmocean

open LLC4320, make a skeleton zarr file for future MLD writing

In [2]:
t_0 = 432
t_1 = t_0 + int(365*24)
LLC = xr.open_zarr('/orcd/data/abodner/003/LLC4320/LLC4320',consolidated=False)[['Theta', 'Salt', 'rA', 'Z','XC','YC']].isel(time=slice(t_0,t_1), face = [7,10])

In [ ]:
# native horizontal sizes
nt = LLC.dims["time"]
nf = LLC.dims["face"]
nj = LLC.dims["j"]
ni = LLC.dims["i"]

# lazy array
MLD_lazy = da.empty(
    shape=(nt, nf, nj, ni),
    chunks=(24, 1, 720, 720),
    dtype=np.float32,
)

MLD_ds = xr.Dataset(
    {
        "MLD": (("time", "face", "j", "i"), MLD_lazy)
    },
    coords={
        "time": LLC.time,
        "face": LLC.face,
        "j": LLC.j,
        "i": LLC.i,
    },
)

compressor = numcodecs.Blosc(cname='zstd', clevel=3, shuffle=numcodecs.Blosc.BITSHUFFLE)

encoding = {
    "MLD": {
        "chunks": (24, 1, 720, 720),
        "compressor": compressor,
        "dtype": "float32",
        "_FillValue": np.nan,
    },
    "time": {"chunks": (nt,), "dtype": "int64"},
    "face": {"chunks": (nf,), "dtype": "int32"},
    "j": {"chunks": (nj,), "dtype": "int32"},
    "i": {"chunks": (ni,), "dtype": "int32"},
}


MLD_ds.to_zarr(
    "/orcd/data/abodner/002/cody/MLD_llc4320/MLD_ds_test.zarr",
    mode="w",
    encoding=encoding,
    consolidated=True,
    zarr_format=2,
    compute=False
)

/tmp/ipykernel_2448188/882740585.py:2: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  nt = LLC.dims["time"]
/tmp/ipykernel_2448188/882740585.py:3: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  nf = LLC.dims["face"]
/tmp/ipykernel_2448188/882740585.py:4: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  nj = LLC.dims["j"]
/tmp/ipykernel_2448188/882740585.py:5: FutureWarning: The return type of `Da

In [ ]:
MLD_test = xr.open_zarr('/orcd/data/abodner/002/cody/MLD_llc4320/MLD_ds_test.zarr',consolidated=True)

In [ ]:
MLD_test